# Model Playground

In [3]:
from pprint import pprint
import numpy as np

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

## convstack function


In [11]:
filters=[8, 16, 16]
kernel_sizes = [(5,5), (3,3), (3,3)]
strides=[(2,2),(1,1),(2,2)]
weight_decay=1e-5

In [46]:
def conv_stack(input_shape:tuple,filters, kernel_sizes, strides, weight_decay, name='conv'):
    assert len(filters) == len(kernel_sizes) \
    and len(filters) == len(strides), "filter/kernel size/stride mismatch"
    
    inputs = layers.Input(name="model_input",
                      shape = input_shape)
    
    Conv2D_1 = layers.Conv2D(name=name+"1",
                             data_format='channels_first',
                             filters=filters[0],
                             kernel_size=kernel_sizes[0],
                             strides=strides[0],
                             kernel_initializer=tf.keras.initializers.GlorotNormal(),
                             activity_regularizer=tf.keras.regularizers.l2(l=weight_decay),
                             activation="relu",
                             )
    BN_1 = layers.BatchNormalization(axis=1, name=name+"1_bn")
    Conv2D_2 = layers.Conv2D(name=name+"2",
                             data_format='channels_first',
                             filters=filters[1],
                             kernel_size=kernel_sizes[1],
                             strides=strides[1],
                             kernel_initializer=tf.keras.initializers.GlorotNormal(),
                             activity_regularizer=tf.keras.regularizers.l2(l=weight_decay),
                             activation="relu",
                             )
    BN_2 = layers.BatchNormalization(axis=1, name=name+"2_bn")
    Conv2D_3 = layers.Conv2D(name=name+"3",
                             data_format='channels_first',
                             filters=filters[2],
                             kernel_size=kernel_sizes[2],
                             strides=strides[2],
                             kernel_initializer=tf.keras.initializers.GlorotNormal(),
                             activity_regularizer=tf.keras.regularizers.l2(l=weight_decay),
                             activation="relu",
                             )
    BN_3 = layers.BatchNormalization(axis=1, name=name+"3_bn")


    stack = Conv2D_1(inputs)
    stack = BN_1(stack)
    stack = Conv2D_2(stack)
    stack = BN_2(stack)
    stack = Conv2D_3(stack)
    conv_output = BN_3(stack)
    
    conv_stack_model = tf.keras.Model(name=name+"_stack",
                       inputs = inputs,
                       outputs = conv_output)
    return conv_stack_model


In [43]:
inputs1 = [tf.Variable(tf.random.uniform([1,1,21,21], -1, 1))]
inputs2 = [tf.Variable(tf.random.uniform([1,1,21,21], 0, 2))]
len(inputs)

1

In [62]:
convstack = conv_stack((1,21,21), filters, kernel_sizes, strides, weight_decay)
enco_in1 = convstack(inputs1)
enco_in1.shape

TensorShape([1, 16, 3, 3])

In [55]:
enco_in2 = convstack(inputs2)

# Encoder function

In [67]:
input_shape_conv = (1,16,3,3)

i=0
input1 = tf.expand_dims(input1, 0)
input1 = layers.Input(name="enco_input{}".format(i+1),
                      shape = input_shape_conv)

encoder_cell_1 = layers.ConvLSTM2D(name="encoder{}".format(i+1),
                                   filters = filters[-1],
                                   kernel_size=(5,5),
                                   padding='same',
                                   data_format='channels_first',
                                   return_sequences=True,
                                   return_state=True)
_, state_h, state_c = encoder_cell_1(input1)

enco_states = [state_h, state_c]
i=1
input2 = tf.expand_dims(input2, 0)
input2 = layers.Input(name="enco_input{}".format(i+1),
                      shape = input_shape_conv)

encoder_cell_2 = layers.ConvLSTM2D(name="encoder{}".format(i+1),
                                   filters = filters[-1],
                                   kernel_size=(5,5),
                                   padding='same',
                                   data_format='channels_first',
                                   return_sequences=True,
                                   return_state=True)
_, state_h, state_c = encoder_cell_1(input2, initial_state=enco_states)

enco_states = [state_h, state_c]


encoder_model = tf.keras.Model(name=name+"_stack",
                       inputs = [input1, input2],
                       outputs = enco_states)

In [68]:
encoder_model([enco_in1, enco_in2])

InvalidArgumentError: transpose expects a vector of size 3. But input(1) is a vector of size 4 [Op:Transpose]

In [69]:
encoder_model.summary()

Model: "convA_stack"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
enco_input1 (InputLayer)        [(None, 1, 16, 3, 3) 0                                            
__________________________________________________________________________________________________
encoder1 (ConvLSTM2D)           [(None, 1, 16, 3, 3) 51264       enco_input1[0][0]                
                                                                 enco_input2[0][0]                
                                                                 encoder1[0][1]                   
                                                                 encoder1[0][2]                   
__________________________________________________________________________________________________
enco_input2 (InputLayer)        [(None, 1, 16, 3, 3) 0                                  